In [4]:
import psycopg2

def connect():
    con = psycopg2.connect("host=" + "localhost" +
                           " port=" + "5432" +
                           " dbname=" + "postgres" +
                           " user=" + "satoki252595" +
                           " password=" + "postgres")
    return con

def select_execute(con, sql):
    with con.cursor() as cur:
        cur.execute(sql)
        rows = cur.fetchall()

    return rows

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report

In [28]:
# データの読み込み

conn = connect()
sql_yahoomap_crawds =  "select * from yahoomap_crawds"
sql_event_details =  "select * from event_details"

yahoomap_crawds = pd.read_sql(sql_yahoomap_crawds,conn)
event_details = pd.read_sql(sql_event_details,conn)

/var/folders/0b/kvbdjp6d6kl4v5f97xnk5h480000gn/T/ipykernel_83010/3523665317.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  yahoomap_crawds = pd.read_sql(sql_yahoomap_crawds,conn)
/var/folders/0b/kvbdjp6d6kl4v5f97xnk5h480000gn/T/ipykernel_83010/3523665317.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  event_details = pd.read_sql(sql_event_details,conn)


In [29]:
# データの前処理
def preprocess_data(df):
    df['date'] = pd.to_datetime(df['date'])
    df['day_of_week'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    return df

yahoomap_crawds = preprocess_data(yahoomap_crawds)
event_details = preprocess_data(event_details)

In [30]:
#重複するカラムを指定
merge_on = ['place', 'date']
# yahoomap_crawdsから使用するカラム
yahoomap_cols = ['place', 'date', 'hour', 'crawds_level']
# event_detailsから使用するカラム（重複するものを除く）
event_cols = [col for col in event_details.columns if col not in merge_on and col != 'crawds_level']

# マージを実行
merged_data = pd.merge(yahoomap_crawds[yahoomap_cols], 
                       event_details[merge_on + event_cols], 
                       on=merge_on, 
                       how='left')

# マージ後のカラムを確認
print("Merged columns:", merged_data.columns)

Merged columns: Index(['place', 'date', 'hour', 'crawds_level', 'id', 'name', 'sub_title',
       'actor', 'open_time', 'start_time', 'close_time', 'time_original',
       'max_price', 'min_price', 'price_text', 'link', 'inquiry', 'detail',
       'num_of_people', 'input_table', 'input_table_rec_id', 'day_of_week',
       'month'],
      dtype='object')


In [36]:
# 特徴量エンジニアリング
merged_data['price_diff'] = merged_data['max_price'] - merged_data['min_price']
merged_data['is_free'] = (merged_data['max_price'] == 0) & (merged_data['min_price'] == 0)

merged_data = preprocess_data(merged_data)

In [37]:
features = ['place', 'hour', 'day_of_week', 'month', 'max_price', 'min_price', 'name']
# 価格差と無料イベントフラグを追加
merged_data['price_diff'] = merged_data['max_price'] - merged_data['min_price']
merged_data['is_free'] = (merged_data['max_price'] == 0) & (merged_data['min_price'] == 0)
features += ['price_diff', 'is_free']

In [38]:
# カテゴリカル変数のエンコーディング
le = LabelEncoder()
merged_data['place'] = le.fit_transform(merged_data['place'])
merged_data['name'] = le.fit_transform(merged_data['name'])

# 'hour' 列を数値に変換
merged_data['hour'] = pd.to_datetime(merged_data['hour'], format='%H:%M').dt.hour

# データ型の確認と変換
for col in merged_data.columns:
    if merged_data[col].dtype == 'object':
        try:
            merged_data[col] = merged_data[col].astype(float)
        except ValueError:
            print(f"列 '{col}' を数値に変換できませんでした。この列を除外します。")
            merged_data = merged_data.drop(col, axis=1)

列 'sub_title' を数値に変換できませんでした。この列を除外します。
列 'open_time' を数値に変換できませんでした。この列を除外します。
列 'start_time' を数値に変換できませんでした。この列を除外します。
列 'time_original' を数値に変換できませんでした。この列を除外します。
列 'price_text' を数値に変換できませんでした。この列を除外します。
列 'link' を数値に変換できませんでした。この列を除外します。
列 'inquiry' を数値に変換できませんでした。この列を除外します。
列 'detail' を数値に変換できませんでした。この列を除外します。
列 'input_table' を数値に変換できませんでした。この列を除外します。


In [39]:
# 特徴量リストの更新
features = [col for col in merged_data.columns if col != 'crawds_level']

# 特徴量と目的変数の準備
X = merged_data[features]
y = merged_data['crawds_level']

In [35]:
# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBMモデルの作成とトレーニング
model = LGBMClassifier(random_state=42)
model.fit(X_train, y_train)

ValueError: pandas dtypes must be int, float or bool.
Fields with bad pandas dtypes: place: object, hour: object, name: object

In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report

# データの読み込み

conn = connect()
sql_yahoomap_crawds =  "select * from yahoomap_crawds"
sql_event_details =  "select * from event_details"

yahoomap_crawds = pd.read_sql(sql_yahoomap_crawds,conn)
event_details = pd.read_sql(sql_event_details,conn)

# 「有明アリーナ」のデータのみを選択
yahoomap_crawds = yahoomap_crawds[yahoomap_crawds['place'] == '有明アリーナ']
event_details = event_details[event_details['place'] == '有明アリーナ']


/var/folders/0b/kvbdjp6d6kl4v5f97xnk5h480000gn/T/ipykernel_83010/1311979772.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  yahoomap_crawds = pd.read_sql(sql_yahoomap_crawds,conn)
/var/folders/0b/kvbdjp6d6kl4v5f97xnk5h480000gn/T/ipykernel_83010/1311979772.py:15: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  event_details = pd.read_sql(sql_event_details,conn)


In [50]:
# データの前処理
def preprocess_data(df):
    df['date'] = pd.to_datetime(df['date'])
    df['day_of_week'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    
    # 'hour' 列の処理を修正
    if 'hour' in df.columns:
        if df['hour'].dtype == 'object':
            # 'hour' が文字列の場合、数値に変換を試みる
            df['hour'] = pd.to_numeric(df['hour'].str.split(':').str[0], errors='coerce')
        else:
            # すでに数値の場合はそのまま使用
            df['hour'] = df['hour'].astype(float)
    
    return df



In [58]:
# 前処理を適用
yahoomap_crawds = preprocess_data(yahoomap_crawds)
event_details = preprocess_data(event_details)

# データの結合
merged_data = pd.merge(yahoomap_crawds, event_details, on=['place', 'date'], how='left', suffixes=('_yahoo', '_event'))

# マージ後にも前処理を適用
merged_data = preprocess_data(merged_data)

# 特徴量エンジニアリング
merged_data['price_diff'] = merged_data['max_price'] - merged_data['min_price']
merged_data['is_free'] = (merged_data['max_price'] == 0) & (merged_data['min_price'] == 0)


In [59]:
# カテゴリカル変数のエンコーディング
le = LabelEncoder()
merged_data['name'] = le.fit_transform(merged_data['name'])

# 特徴量の選択
features = ['hour', 'day_of_week', 'month', 'max_price', 'min_price', 'price_diff', 'is_free', 'name']

# 利用可能な特徴量のみを使用
available_features = [f for f in features if f in merged_data.columns]
X = merged_data[available_features]
y = merged_data['crawds_level']

print("使用する特徴量:", available_features)

使用する特徴量: ['hour', 'day_of_week', 'month', 'max_price', 'min_price', 'price_diff', 'is_free', 'name']


In [60]:
# 特徴量の選択
features = ['hour', 'day_of_week', 'month', 'max_price', 'min_price', 'price_diff', 'is_free', 'name']
X = merged_data[features]
y = merged_data['crawds_level']

print("データ型:")
print(X.dtypes)
print("\n使用する特徴量:", features)



データ型:
hour           float64
day_of_week      int32
month            int32
max_price      float64
min_price      float64
price_diff     float64
is_free           bool
name             int64
dtype: object

使用する特徴量: ['hour', 'day_of_week', 'month', 'max_price', 'min_price', 'price_diff', 'is_free', 'name']


In [62]:
# データの分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBMモデルの作成とトレーニング
model = LGBMClassifier(random_state=42)
model.fit(X_train, y_train)

# モデルの評価
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAccuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# 特徴量の重要度
feature_importance = pd.DataFrame({'feature': features, 'importance': model.feature_importances_})
feature_importance = feature_importance.sort_values('importance', ascending=False)
print("\nFeature Importance:")
print(feature_importance)



[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000321 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 56
[LightGBM] [Info] Number of data points in the train set: 1654, number of used features: 7
[LightGBM] [Info] Start training from score -0.126817
[LightGBM] [Info] Start training from score -3.148272
[LightGBM] [Info] Start training from score -2.574670
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furt

In [ ]:
# 新しいデータに対する予測
new_data = pd.DataFrame({
    'date': ['2024-07-01'],
    'hour': ['19:00'],
    'name': ['新しいイベント'],
    'max_price': [12000],
    'min_price': [6000]
})

new_data = preprocess_data(new_data)
new_data['name'] = le.transform(['新しいイベント'])  # 新しいイベント名をエンコード
new_data['price_diff'] = new_data['max_price'] - new_data['min_price']
new_data['is_free'] = (new_data['max_price'] == 0) & (new_data['min_price'] == 0)

new_prediction = model.predict(new_data[features])
print(f"\n新しいイベントの予測混雑度: {new_prediction[0]}")